# teste

In [1]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

# Configuração do dispositivo (verificar uso de GPU)
print("TensorFlow version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TensorFlow version: 2.4.1
GPU is available


In [2]:
# Test data directory
TEST_DIR = '../data/test'
batch_size = 8
input_shape = (224, 224, 3)

# Define ImageDataGenerator for test set (no data augmentation, just preprocessing)
test_datagen_resTS = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input)
test_datagen_efficientnet = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)

# Load test data
test_generator_resTS = test_datagen_resTS.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator_efficientnet = test_datagen_efficientnet.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 8179 images belonging to 38 classes.
Found 8179 images belonging to 38 classes.


In [3]:
# Load ResTS model
ResTS_model = tf.keras.models.load_model('../models/ResTS_trained_model')

# Load EfficientNet-B0 model and load saved weights
EfficientNetB0_model = tf.keras.models.load_model('../models/EfficientNetB0_trained_model.h5')

In [4]:
# Evaluation function for a dual-output model (ResTeacher and ResStudent)
def evaluate_dual_output_model(model, generator):
    start_time = time.time()
    predictions = model.predict(generator, steps=len(generator))
    end_time = time.time()

    # Separate outputs for ResTeacher and ResStudent
    y_pred_teacher = np.argmax(predictions[0], axis=1)
    y_pred_student = np.argmax(predictions[1], axis=1)
    y_true = generator.classes

    # Evaluation Metrics for ResTeacher
    accuracy_teacher = accuracy_score(y_true, y_pred_teacher)
    f1_teacher = f1_score(y_true, y_pred_teacher, average='weighted')
    
    # Evaluation Metrics for ResStudent
    accuracy_student = accuracy_score(y_true, y_pred_student)
    f1_student = f1_score(y_true, y_pred_student, average='weighted')

    processing_time = end_time - start_time

    return {
        "ResTeacher": (accuracy_teacher, f1_teacher),
        "ResStudent": (accuracy_student, f1_student),
        "processing_time": processing_time,
        "y_true": y_true,
        "y_pred_teacher": y_pred_teacher,
        "y_pred_student": y_pred_student
    }

# Evaluation function for a single-output model (EfficientNet-B0)
def evaluate_single_output_model(model, generator):
    start_time = time.time()
    predictions = model.predict(generator, steps=len(generator))
    end_time = time.time()

    y_pred = np.argmax(predictions, axis=1)
    y_true = generator.classes

    # Evaluation Metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')

    processing_time = end_time - start_time

    return accuracy, f1, processing_time, y_true, y_pred

# Evaluate ResTS model
print("\nEvaluating ResTS Model")
results_resTS = evaluate_dual_output_model(ResTS_model, test_generator_resTS)

# Evaluate EfficientNet-B0 model
print("\nEvaluating EfficientNet-B0 Model")
eff_accuracy, eff_f1, eff_time, y_true, y_pred_eff = evaluate_single_output_model(EfficientNetB0_model, test_generator_efficientnet)


Evaluating ResTS Model

Evaluating EfficientNet-B0 Model


In [6]:
# Summary of the results:
results_summary = pd.DataFrame({
    "Model": ["ResTeacher", "ResStudent", "EfficientNetB0"],
    "Accuracy": [results_resTS["ResTeacher"][0], results_resTS["ResStudent"][0], eff_accuracy],
    "F1 Score": [results_resTS["ResTeacher"][1], results_resTS["ResStudent"][1], eff_f1],
    "Processing Time (s)": [results_resTS["processing_time"], results_resTS["processing_time"], eff_time]
})

print("\nSummary of Evaluation Results")
print(results_summary)


Summary of Evaluation Results
            Model  Accuracy  F1 Score  Processing Time (s)
0      ResTeacher  0.964177  0.963888           127.490549
1      ResStudent  0.961731  0.961800           127.490549
2  EfficientNetB0  0.954884  0.955838            29.412510
